# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
### Lab 3. Object detection 모델을 AWS DeepLens에 배포하기

<font color='red'>Lab 3은 제공된 AWS 계정을 이용해서 그룹이 함께 수행합니다</font>

#### 랩 순서

1. 제공된 USB에 있는 모델 파일을 각 그룹의 S3 버킷에 업로드합니다.
2. DeepLens 모델 정의하기
3. 제공된 DeepLens 프로젝트에서 모델 교체하기
4. DeepLens에 배포하고 테스트하기

DeepLens에 배포한 모델을 저장할 S3 버킷이름과 모델 파일 이름을 정합니다.

> 주의: S3 버킷 이름은 꼭 **"deeplens-"**로 시작해야합니다.

In [1]:
S3_BUCKET = 'deeplens-devday-seoul-hol-group1' # 각 그룹 번호 (1-8)으로 바꿔주세요.
DL_MODEL_NAME = 'model-to-go.tar.gz'

### Task 1 - 제공된 USB에 있는 모델 파일을 각 그룹의 S3 버킷에 업로드합니다.

1. 주피터 노트북 왼쪽 메뉴의 '업로드' 아이콘을 클릭해서 USB에 있는 'model-to-go.tar.gz'를 노트북 인스턴스로 업로드합니다.

![](./images/devday-deeplens-lab-12.png)

2. 다음 명령을 수행해서 S3로 업로드 합니다.

In [8]:
!aws s3 cp ./$DL_MODEL_NAME s3://$S3_BUCKET/models/

./deeplens-yolov3-0000.params
./deeplens-yolov3-symbol.json
upload: ./model-yolov3.tar.gz to s3://deeplens-mh-s3-us-east-1/models/model-yolov3.tar.gz


### Task 2 - DeepLens 모델 정의하기

> **주의** 강연자의 별도의 안내가 있을 경우, Task 2를 수행하지 않고 Task 3을 수행합니다.

1. DeepLens 콜솔로 이동 후, 'Resources > Models' 선택
2. 'Import model' 버튼 클릭

![](./images/devday-deeplens-lab-01.png)

3. 'Import source'를 'Externally trained model'로 변경
4. Model settings에 값들을 입력
- Model artifact path: s3://__S3_BUCKET__/models/model-yolov3.tar.gz
- Model name: model 이름 (예, object-detection)
- Model framework: MXNet 선택

![](./images/devday-deeplens-lab-02.png)

### Task 3 - 제공된 DeepLens 프로젝트에서 모델 교체하기

1. DeepLens 콜솔로 이동 후, 'Resources > Projects' 선택하고,
2. 프로젝트 목록에서 'devday' 클릭합니다.

![](./images/devday-deeplens-lab-13.png)

3. 'Edit' 클릭 후, 모델 변경을 위해서 Model의 'Remove' 버튼 클릭

![](./images/devday-deeplens-lab-08.png)

4. GluonCV로 학습시킨 모델을 추가하기 위해서, 'Add model' 클릭

![](./images/devday-deeplens-lab-09.png)

5. 모델 목록에서, 이전 단계에서 정의한 모델 선택 후 'Add model' 버튼 클ㄹ릭

![](./images/devday-deeplens-lab-14.png)

6. 프로젝트를 DeepLens에 배포하기 위해서, 프로젝트 목록에서 Object-detection을 선택한 후, 'Deploy to Device'를 클릭합니다.

![](./images/devday-deeplens-lab-15.png)

7. 'Target device' 목록에서 본인의 그룹 디바이스를 선택하고, 'Review'를 클릭합니다. 다음 화면에서 'Deploy'를 클릭합니다.

![](./images/devday-deeplens-lab-16.png)

8. DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래 명령을 수행합니다.

```
mplayer -demuxer lavf /opt/awscam/out/ch1_out.h264
```

![](./images/devday-deeplens-lab-16.png)

9. 프로젝트 배포는 위 상태창에 나오고 완료되면 노색 상태 메시지창으로 바뀝니다.

![](./images/devday-deeplens-lab-17.png)
![](./images/devday-deeplens-lab-18.png)

10. 수행 결과 확인

DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래 명령을 수행합니다.

```
mplayer -demuxer lavf /opt/awscam/out/ch1_out.h264
```

### 참고

* Greengrass 서비스 재시작

```
sudo systemctl restart greengrassd.service
```